<a href="https://colab.research.google.com/github/ciccio1982/Colab/blob/main/inviare_messaggio_su_Solace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install solace-pubsubplus

First, you need to import the necessary libraries and configure your Solace connection details. Replace the placeholder values with your actual Solace broker's IP address or hostname, VPN name, username, and password.

In [9]:
import os
import time

# Import Solace Python  API modules
from solace.messaging.messaging_service import MessagingService, ReconnectionListener, ReconnectionAttemptListener, ServiceInterruptionListener, RetryStrategy, ServiceEvent
from solace.messaging.errors.pubsubplus_client_error import PubSubPlusClientError
from solace.messaging.publisher.direct_message_publisher import PublishFailureListener, FailedPublishEvent
from solace.messaging.resources.topic_subscription import TopicSubscription
from solace.messaging.receiver.message_receiver import MessageHandler
from solace.messaging.config.solace_properties.message_properties import APPLICATION_MESSAGE_ID
from solace.messaging.resources.topic import Topic
from solace.messaging.receiver.inbound_message import InboundMessage
# Solace Connection Details (replace with your actual values)
SOLACE_HOST = 'tcps://mr-connection-1d0n94c2091.messaging.solace.cloud:55443' # Changed to tcp:// for non-TLS
SOLACE_VPN = 'myfirstservice' # e.g., default
SOLACE_USERNAME = 'solace-cloud-client' # e.g., guest
SOLACE_PASSWORD = 'nj8nmnunpa68tg00rfa06hol8r' # e.g., guest

# Queue to send messages to
TARGET_QUEUE = 'test1'

In [10]:
# Define broker_props using the already defined SOLACE_* variables
broker_props = {
    "solace.messaging.transport.host": SOLACE_HOST,
    "solace.messaging.service.vpn-name": SOLACE_VPN,
    "solace.messaging.authentication.scheme.basic.username": SOLACE_USERNAME,
    "solace.messaging.authentication.scheme.basic.password": SOLACE_PASSWORD,
    # Disabilita validazione certificati
    "solace.messaging.tls.cert-validated": False,
    "solace.messaging.tls.cert-reject-expired": False
    # TLS-specific properties are removed for tcp:// connection
}

# Create the messaging service using from_properties
messaging_service = MessagingService.builder() \
  .from_properties(broker_props) \
  .with_reconnection_retry_strategy(RetryStrategy.parametrized_retry(20,3)) \
  .build()

messaging_service.connect()
print("CONNESSO!!!")

CONNESSO!!!


Now, let's create a publisher, connect to the Solace broker, and send a simple text message to the queue. The `solace-pubsub-qs-python` library simplifies the process of sending messages.

In [11]:
try:
    # Create a direct message publisher
    publisher = messaging_service.create_direct_message_publisher_builder().build()
    publisher.start()
    print("Publisher started successfully.")

    # Message to send
    message_payload = "Hello from Google Colab! This is a test message."

    # Build the message
    message = messaging_service.message_builder() \
        .with_property(APPLICATION_MESSAGE_ID, "my-message-id-1") \
        .build(message_payload)

    # Publish the message to the target queue as a topic
    # For queues, Solace uses a topic dispatch type to route messages
    publisher.publish(message, Topic.of(TARGET_QUEUE))
    print(f"Sent message: '{message_payload}' to queue '{TARGET_QUEUE}'.")

except PubSubPlusClientError as e:
    print(f"PubSubPlusClientError: {e}")
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    if 'publisher' in locals() and publisher.is_running():
        publisher.terminate()
        print("Publisher terminated.")
    if messaging_service.is_connected:
        messaging_service.disconnect()
        print("Disconnected from Solace.")

Publisher started successfully.
Sent message: 'Hello from Google Colab! This is a test message.' to queue 'test1'.
Publisher terminated.
Disconnected from Solace.


Remember to replace the placeholder values in the `SOLACE_HOST`, `SOLACE_VPN`, `SOLACE_USERNAME`, and `SOLACE_PASSWORD` variables with your actual Solace broker credentials and network details. Also, ensure the `TARGET_QUEUE` exists on your Solace broker.